In [3]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../../ukb-dementia-shap/")



from logic.data_processing.data_setup import *
from logic.analysis.analysis import AnalysisCharts
from pandas.tseries.offsets import DateOffset

ds=data_setup()
an=AnalysisCharts()
dp=data_proc_main()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
df_brain=pd.read_parquet('../data/ukb_brain.parquet')
df_model=pd.read_parquet(dp.path+'df_all_final2022-04-13.parquet')
mask=pd.notnull(df_brain['volume_of_white_matter_f25008_2_0'])
fields=pd.read_excel('../data/ukb_field_names.xlsx',sheet_name="fieldnames_full")
brain_fields_andrew=pd.read_csv('../data/brain_fields_andrew.csv')

brain_cols_and=list(brain_fields_andrew['features'])+['volumetric_scaling_from_t1_head_image_to_standard_space_f25000_2_0']
mask=pd.notnull(fields['field_type2'])&(fields['field_type2'].str.contains('Brain'))
fields.loc[mask,]
braincols=['eid']+list(fields.loc[mask,'col.name'].unique())

mask=pd.notnull(df_brain['volumetric_scaling_from_t1_head_image_to_standard_space_f25000_2_0'])
df_brain2=df_brain.loc[mask,]
df_brain2['eid']=df_brain2['eid'].astype(str)
mri_eids=list(df_brain2['eid'])
#braincols2=[c for c in braincols if 'eid' in c or ( c[len(c)-3:len(c)]=='2_0' and 'volume' in c)]
braincols2=[c for c in df_brain2.columns if c in brain_cols_and or 'eid' in c]
df_brain_full=pd.merge(df_brain2[braincols2],df_model,on='eid',how='inner')
df_brain_mri_only=pd.merge(df_brain2[braincols2],df_model[['eid','age_when_attended_assessment_centre_f21003_0_0','sex_f31_0_0']],on='eid',how='inner')

multvar='volumetric_scaling_from_t1_head_image_to_standard_space_f25000_2_0'
for v in df_brain_mri_only.columns:
    if v!='eid' and 'sex' not in v and 'age_when' not in v and 'volume' in v and v!=multvar and 'norm' :
        df_brain_mri_only[v]=df_brain_mri_only[v]*df_brain_mri_only[multvar]

df_brain_mri_only.to_parquet('../data/df_brain_mri.parquet')